In [1]:
import os
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "ib_workflows_backend.settings.local")

django.setup()
from elasticsearch_dsl import connections
from django.conf import settings 
connections.create_connection(hosts=[settings.ELASTICSEARCH_ENDPOINT],timeout=20)
from elasticsearch_dsl import Document, Date, Nested, Boolean, \
    analyzer, InnerDoc, Completion, Keyword, Text, Q, Search, Mapping, Integer, Keyword, A

In [2]:
project_id ="FIN_MAN"
adhoc_template_id = "FIN_PR"

In [3]:
from elasticsearch_dsl import Q, Search
query = Q("term", project_id__keyword=project_id) \
            & Q('term', template_id__keyword=adhoc_template_id)

In [4]:
query

Bool(must=[Term(project_id__keyword='FIN_MAN'), Term(template_id__keyword='FIN_PR')])

In [5]:
search = Search(index="task-local")

In [6]:
search


In [7]:
search.to_dict()

{}

In [8]:
field_agg = A('terms', field='FIN_PURPOSE_OF_THE_ORDER.keyword')
stage_agg = A('terms', field='stages.stage_id.keyword')
a_data = A('top_hits')
search.aggs.bucket('groups',field_agg).bucket('child_groups', stage_agg).bucket('tasks', a_data)

TopHits()

In [9]:
search.to_dict()

{'aggs': {'groups': {'terms': {'field': 'FIN_PURPOSE_OF_THE_ORDER.keyword'},
   'aggs': {'child_groups': {'terms': {'field': 'stages.stage_id.keyword'},
     'aggs': {'tasks': {'top_hits': {}}}}}}}}

In [10]:
response = search.execute()

[none] [ib_workflows_backend - local] INFO     [16:17:22]  [/home/ib-developer/PycharmProjects/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/task-local/_search [status:200 request:0.013s]


In [15]:
for group in response.aggregations.groups.buckets:
    print("group_count", group.doc_count)
    print("grouped_by", group.key)
    print("group")
    for child_group in group.child_groups:
#         print(task.value)
        print("chld_group")
        print("child_grouped_by", child_group.key)
        print("child_group_count", child_group.doc_count)
        for task in child_group.tasks:
            print(task.task_id)
        print("************")
    print("**********")


group_count 1
grouped_by need to pay debt
group
chld_group
child_grouped_by PR_PAYMENT_REQUEST_DRAFTS
child_group_count 1
19
************
**********
group_count 1
grouped_by need to pay friend
group
chld_group
child_grouped_by PR_PAYMENT_REQUEST_DRAFTS
child_group_count 1
20
************
**********
group_count 1
grouped_by purpose
group
chld_group
child_grouped_by PR_NEED_CLARIFICATION
child_group_count 1
24
************
**********
group_count 1
grouped_by sfsdd
group
chld_group
child_grouped_by PR_PAYMENT_REQUEST_DRAFTS
child_group_count 1
21
************
**********
group_count 1
grouped_by sfsdfsd
group
chld_group
child_grouped_by PR_PAYMENT_REQUEST_DRAFTS
child_group_count 1
25
************
**********


In [13]:
response.aggregations.groups.buckets

[{'key': 'need to pay debt', 'doc_count': 1, 'child_groups': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'PR_PAYMENT_REQUEST_DRAFTS', 'doc_count': 1, 'tasks': {'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'task-local', '_type': '_doc', '_id': 'AQtuoHQBlEpf_YrJx9SP', '_score': 1.0, '_source': {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id': 19, 'title': 'draft task', 'FIN_TYPE_OF_PAYMENT_REQUEST': 'Expense Card Loading', 'FIN_EVENTS_ACTIVITY_PETTYCASH': 'iB Hubs Pool Drive (Vizag, East & West Godavari)', 'FIN_OFFICE_LOCATION_FOR_WHICH_PURCHASE_IS_BEING_DONE': '30', 'FIN_PURPOSE_OF_THE_ORDER': 'need to pay debt', 'stages': [{'stage_id': 'PR_PAYMENT_REQUEST_DRAFTS'}]}}]}}}]}}, {'key': 'need to pay friend', 'doc_count': 1, 'child_groups': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 0, 'buckets': [{'key': 'PR_PAYMENT_REQUEST_DRAFTS', 'doc_count': 1, 'tasks': {'hits': {'total': {'value'

In [248]:
# total tasks

In [249]:
from elasticsearch_dsl import Q, Search
query = Q("term", project_id__keyword=project_id) \
            & Q('term', template_id__keyword=adhoc_template_id)

In [284]:
search = Search(index="task-local")

In [251]:
search.to_dict()

{}

In [252]:
task_objects = search.filter(query)

In [253]:
response = search.execute()

In [272]:
for task in task_objects:
    print(task.task_id)

19
20
21
22
23
24
25
26
27
28


In [227]:
task_objects.count()

19

In [270]:
task_objects[1:5]

In [303]:
# For one grouping

In [11]:
from elasticsearch_dsl import Q, Search
query = Q("term", project_id__keyword=project_id) \
            & Q('term', template_id__keyword=adhoc_template_id)

In [16]:
search = Search(index="task-local")
search.filter(query)
field_agg = A('terms', field='FIN_PURPOSE_OF_THE_ORDER.keyword')
tasks_data = A('top_hits')

In [17]:
search.aggs.bucket('groups',field_agg).bucket('tasks', tasks_data)

TopHits()

In [18]:
response = search.execute()

[none] [ib_workflows_backend - local] INFO     [15:29:36]  [/home/ib-developer/PycharmProjects/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST http://localhost:9200/task-local/_search [status:200 request:0.003s]


In [19]:
search.to_dict()

{'aggs': {'groups': {'terms': {'field': 'FIN_PURPOSE_OF_THE_ORDER.keyword'},
   'aggs': {'tasks': {'top_hits': {}}}}}}

In [21]:
response = search.execute()

In [30]:
len(response.aggregations.groups.buckets)

5

In [35]:
for group in response.aggregations.groups.buckets:
    print(group.key)
    print(group.doc_count)
    print(group.tasks)

need to pay debt
1
<Response: [<Hit(task-local/AQtuoHQBlEpf_YrJx9SP): {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id'...}>]>
need to pay friend
1
<Response: [<Hit(task-local/HQtvoHQBlEpf_YrJstTD): {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id'...}>]>
purpose
1
<Response: [<Hit(task-local/ngu-oHQBlEpf_YrJAdTi): {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id'...}>]>
sfsdd
1
<Response: [<Hit(task-local/QwuIoHQBlEpf_YrJ_tRM): {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id'...}>]>
sfsdfsd
1
<Response: [<Hit(task-local/zgvAoHQBlEpf_YrJJdSg): {'project_id': 'FIN_MAN', 'template_id': 'FIN_PR', 'task_id'...}>]>
